In [1]:
import os
from google.colab import userdata
import json

In [2]:
# with open("kaggle.json", "r") as file:
#     keys = json.load(file)

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [6]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

In [4]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [5]:
import keras
import keras_nlp

In [ ]:
#!wget -O databricks-dolly-15k.jsonl https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl

--2024-05-06 05:51:29--  https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
Resolving huggingface.co (huggingface.co)... 18.172.134.4, 18.172.134.88, 18.172.134.124, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/34/ac/34ac588cc580830664f592597bb6d19d61639eca33dc2d6bb0b6d833f7bfd552/2df9083338b4abd6bceb5635764dab5d833b393b55759dffb0959b6fcbf794ec?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27databricks-dolly-15k.jsonl%3B+filename%3D%22databricks-dolly-15k.jsonl%22%3B&Expires=1715233889&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNTIzMzg4OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zNC9hYy8zNGFjNTg4Y2M1ODA4MzA2NjRmNTkyNTk3YmI2ZDE5ZDYxNjM5ZWNhMzNkYzJkNmJiMGI2ZDgzM2Y3YmZkNTUyLzJkZjkwODMzMzhiNGFiZDZiY2ViNTYzNTc2NGRhY

In [12]:
# import json
# data = []
# with open("astro.jsonl") as file:
#     for line in file:
#         features = json.loads(line)
#         # Filter out examples with context, to keep it simple.
#         if features["context"]:
#             continue
#         # Format the entire example as a single string.
#         template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
#         data.append(template.format(**features))

# # Only use 1000 training examples, to keep it fast.
# data = data[:1000]
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

data = []

with open("astro.jsonl") as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples without context to keep it simple.
        if not features["context"]:
            continue

        # Create instruction and response fields.
        instruction = "What is the usage of the word '{}' in the context of astronomy?".format(features["target"])
        responses = features["knowledge"]
        for response in responses:
            formatted_example = template.format(instruction=instruction, response=response)
            data.append(formatted_example)

data[:5]

["Instruction:\nWhat is the usage of the word 'abound' in the context of astronomy?\n\nResponse:\nStars abound in the Milky Way galaxy.",
 "Instruction:\nWhat is the usage of the word 'abound' in the context of astronomy?\n\nResponse:\nTheories about dark matter abound in modern astrophysics.",
 "Instruction:\nWhat is the usage of the word 'abound' in the context of astronomy?\n\nResponse:\nGas and dust particles abound in the interstellar medium.",
 "Instruction:\nWhat is the usage of the word 'abound' in the context of astronomy?\n\nResponse:\nObservations abound of exoplanets in our galaxy.",
 "Instruction:\nWhat is the usage of the word 'abound' in the context of astronomy?\n\nResponse:\nNeutron stars abound in the dense regions of the galaxy."]

In [8]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'task.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'metadata.json' from model 

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [13]:
prompt = template.format(
    instruction="What is the usage of the word 'abound' in the context of astronomy?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What is the usage of the word 'abound' in the context of astronomy?

Response:
'Abundance' refers to the relative amount of a particular element or other property.

'Abundance' is used in the context of astronomy to mean 'a large amount of a specific chemical element or compound'.
Abundance is used to describe the concentration of elements in the universe as a whole, and is a useful measure of the relative amount of a given element or compound in the cosmos.
Abundance of a specific element in the Universe is determined by a number of factors, including the initial conditions of the universe, the rate of chemical reactions and the rate of cosmic ray bombardment.
The abundance of a specific element or compound can be used to infer the age and history of the Universe.
The abundance of a given chemical element can also be used to infer the presence of life on Earth or other planets.
The abundance of a chemical element can be determined by analyzing a sample of the substance or

In [14]:
# prompt = template.format(
#     instruction="Explain the process of photosynthesis in a way that a child could understand.",
#     response="",
# )
# print(gemma_lm.generate(prompt, max_length=256))

In [15]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [16]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

240/240 ━━━━━━━━━━━━━━━━━━━━ 384s 1s/step - loss: 0.2190 - sparse_categorical_accuracy: 0.4981


In [17]:
prompt = template.format(
    instruction="What is the usage of the word 'abound' in the context of astronomy?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What is the usage of the word 'abound' in the context of astronomy?

Response:
A plethora of planets has abounded in the Milky Way galaxy.


In [18]:
prompt = template.format(
    instruction="What is the usage of the word 'congenial' in the context of astronomy?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What is the usage of the word 'congenial' in the context of astronomy?

Response:
A congenial environment can encourage astronomers to explore the universe.


In [22]:
def inference_generator(prompt):
    sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
    gemma_lm.compile(sampler=sampler)
    print(gemma_lm.generate(prompt, max_length=256))

list_of_words = ['disseminate',
 'feasible',
 'flout',
 'homogeneous',
 'humdrum',
 'insipid',
 'loquacious',
 'misanthropic',
 'misnomer',
 'negligent',
 'obsequious',
 'placate',
 'proclivity',
 'puerile',
 'quixotic',
 'spendthrift',
 'taciturn',
 'wary']

for word in list_of_words:
    prompt = template.format(
        instruction="What is the usage of the word '{}' in the context of astronomy?".format(word),
        response="",
    )
    inference_generator(prompt)
    print("--"*25)


Instruction:
What is the usage of the word 'disseminate' in the context of astronomy?

Response:
To disseminate the information of a new star formation region is a major step toward understanding how these objects form and evolve.
--------------------------------------------------
Instruction:
What is the usage of the word 'feasible' in the context of astronomy?

Response:
A feasible observation program requires careful planning to avoid conflicts with other astronomical observations.
--------------------------------------------------
Instruction:
What is the usage of the word 'flout' in the context of astronomy?

Response:
Scientists flouted the rules of the solar system by claiming that Pluto was actually a planet.
--------------------------------------------------
Instruction:
What is the usage of the word 'homogeneous' in the context of astronomy?

Response:
A homogeneous star exhibits no significant variation in its chemical composition across its surface.
------------------------

## Testing Unseen Words

In [23]:
unseen_words = ["benign", "brazen", "calumny", "candid", "demur","incredulous"]

for word in unseen_words:
    prompt = template.format(
        instruction="What is the usage of the word '{}' in the context of astronomy?".format(word),
        response="",
    )
    inference_generator(prompt)
    print("--"*25)


Instruction:
What is the usage of the word 'benign' in the context of astronomy?

Response:
Astronomers use the term benign to describe the relatively peaceful nature of the early universe.
--------------------------------------------------
Instruction:
What is the usage of the word 'brazen' in the context of astronomy?

Response:
Although the spacecraft's instruments were highly sophisticated, the data they produced were often brazenly incorrect.
--------------------------------------------------
Instruction:
What is the usage of the word 'calumny' in the context of astronomy?

Response:
A spate of scientific research on the cosmos has been marred by the dissemination of inaccurate or outright false information. This has been attributed to the phenomenon of calunny among scientists.
--------------------------------------------------
Instruction:
What is the usage of the word 'candid' in the context of astronomy?

Response:
In the study of stars, a candid approach to the data can help 